# Desafio 6

Neste desafio, vamos praticar _feature engineering_, um dos processos mais importantes e trabalhosos de ML. Utilizaremos o _data set_ [Countries of the world](https://www.kaggle.com/fernandol/countries-of-the-world), que contém dados sobre os 227 países do mundo com informações sobre tamanho da população, área, imigração e setores de produção.

> Obs.: Por favor, não modifique o nome das funções de resposta.

## _Setup_ geral

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk
import time
from sklearn.preprocessing import KBinsDiscretizer, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [6]:
## Algumas configurações para o matplotlib.
#%matplotlib inline

#from IPython.core.pylabtools import figsize


#figsize(12, 8)

#sns.set()

In [7]:
countries = pd.read_csv("data/countries.csv", decimal=',')

In [8]:
new_column_names = [
    "Country", "Region", "Population", "Area", "Pop_density", "Coastline_ratio",
    "Net_migration", "Infant_mortality", "GDP", "Literacy", "Phones_per_1000",
    "Arable", "Crops", "Other", "Climate", "Birthrate", "Deathrate", "Agriculture",
    "Industry", "Service"
]

countries.columns = new_column_names

countries.head(5)

,Country,Region,Population,Area,Pop_density,Coastline_ratio,Net_migration,Infant_mortality,GDP,Literacy,Phones_per_1000,Arable,Crops,Other,Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,48.0,0.00,23.06,163.07,700.0,36.0,3.2,12.13,0.22,87.65,1.0,46.60,20.34,0.380,0.240,0.380
1,Albania,EASTERN EUROPE,3581655,28748,124.6,1.26,-4.93,21.52,4500.0,86.5,71.2,21.09,4.42,74.49,3.0,15.11,5.22,0.232,0.188,0.579
2,Algeria,NORTHERN AFRICA,32930091,2381740,13.8,0.04,-0.39,31.00,6000.0,70.0,78.1,3.22,0.25,96.53,1.0,17.14,4.61,0.101,0.600,0.298
3,American Samoa,OCEANIA,57794,199,290.4,58.29,-20.71,9.27,8000.0,97.0,259.5,10.00,15.00,75.00,2.0,22.46,3.27,NaN,NaN,NaN
4,Andorra,WESTERN EUROPE,71201,468,152.1,0.00,6.60,4.05,19000.0,100.0,497.2,2.22,0.00,97.78,3.0,8.71,6.25,NaN,NaN,NaN


## Observações

Esse _data set_ ainda precisa de alguns ajustes iniciais. Primeiro, note que as variáveis numéricas estão usando vírgula como separador decimal e estão codificadas como strings. Corrija isso antes de continuar: transforme essas variáveis em numéricas adequadamente.

Além disso, as variáveis `Country` e `Region` possuem espaços a mais no começo e no final da string. Você pode utilizar o método `str.strip()` para remover esses espaços.

## Inicia sua análise a partir daqui

In [9]:
countries.shape

(227, 20)

In [10]:
# Sua análise começa aqui.
countries.dtypes

Country              object
Region               object
Population            int64
Area                  int64
Pop_density         float64
Coastline_ratio     float64
Net_migration       float64
Infant_mortality    float64
GDP                 float64
Literacy            float64
Phones_per_1000     float64
Arable              float64
Crops               float64
Other               float64
Climate             float64
Birthrate           float64
Deathrate           float64
Agriculture         float64
Industry            float64
Service             float64
dtype: object

In [11]:
countries['Country'] = countries['Country'].str.strip()
countries['Region'] = countries['Region'].str.strip()
countries['Country'].to_list()[:5]

['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra']

In [12]:
numeric_columns = countries.columns[2:]
countries[numeric_columns].apply(pd.to_numeric)
countries.dtypes

Country              object
Region               object
Population            int64
Area                  int64
Pop_density         float64
Coastline_ratio     float64
Net_migration       float64
Infant_mortality    float64
GDP                 float64
Literacy            float64
Phones_per_1000     float64
Arable              float64
Crops               float64
Other               float64
Climate             float64
Birthrate           float64
Deathrate           float64
Agriculture         float64
Industry            float64
Service             float64
dtype: object

In [13]:
t1 = time.time()
sorted([*countries['Region'].unique()])
time.time() - t1

0.0009796619415283203

In [14]:
t2 = time.time()
[*np.sort(countries['Region'].unique())]
time.time() - t2

0.0

In [15]:
kBins = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
bins = kBins.fit_transform(countries[['Pop_density']])
df1 = pd.concat([countries['Pop_density'], pd.DataFrame(bins, columns=['intervalo'])], axis=1)
df1[df1['intervalo'] >= 9]['intervalo'].count()

23

In [16]:
len(bins)

227

In [17]:
countries[['Region', 'Climate']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Region   227 non-null    object 
 1   Climate  205 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.7+ KB


## Questão 1

Quais são as regiões (variável `Region`) presentes no _data set_? Retorne uma lista com as regiões únicas do _data set_ com os espaços à frente e atrás da string removidos (mas mantenha pontuação: ponto, hífen etc) e ordenadas em ordem alfabética.

In [41]:
def q1():
    # Retorne aqui o resultado da questão 1.
    return sorted([*countries['Region'].unique()])

In [42]:
q1()

['ASIA (EX. NEAR EAST)',
 'BALTICS',
 'C.W. OF IND. STATES',
 'EASTERN EUROPE',
 'LATIN AMER. & CARIB',
 'NEAR EAST',
 'NORTHERN AFRICA',
 'NORTHERN AMERICA',
 'OCEANIA',
 'SUB-SAHARAN AFRICA',
 'WESTERN EUROPE']

## Questão 2

Discretizando a variável `Pop_density` em 10 intervalos com `KBinsDiscretizer`, seguindo o encode `ordinal` e estratégia `quantile`, quantos países se encontram acima do 90º percentil? Responda como um único escalar inteiro.

In [20]:
def q2():
    # Retorne aqui o resultado da questão 2.
    kBins = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
    bins = kBins.fit_transform(countries[['Pop_density']])
    df1 = pd.concat([countries['Pop_density'], pd.DataFrame(bins, columns=['intervalo'])], axis=1)
    return df1[df1['intervalo'] >= 9]['intervalo'].count()

# Questão 3

Se codificarmos as variáveis `Region` e `Climate` usando _one-hot encoding_, quantos novos atributos seriam criados? Responda como um único escalar.

In [21]:
def q3():
    # Retorne aqui o resultado da questão 3.
    oneHot = OneHotEncoder(sparse=False)
    dfOneHotEnc = oneHot.fit_transform(countries[['Region', 'Climate']].fillna(value=countries['Climate'].mean()))
    return dfOneHotEnc.shape[1]

In [22]:
oneHot = OneHotEncoder(sparse=False)
dfOneHotEnc = oneHot.fit_transform(countries[['Region', 'Climate']].fillna(value=countries['Climate'].mean()))
dfOneHotEnc.shape[1]

18

In [23]:
q3()

18

## Questão 4

Aplique o seguinte _pipeline_:

1. Preencha as variáveis do tipo `int64` e `float64` com suas respectivas medianas.
2. Padronize essas variáveis.

Após aplicado o _pipeline_ descrito acima aos dados (somente nas variáveis dos tipos especificados), aplique o mesmo _pipeline_ (ou `ColumnTransformer`) ao dado abaixo. Qual o valor da variável `Arable` após o _pipeline_? Responda como um único float arredondado para três casas decimais.

In [24]:
test_country = [
    'Test Country', 'NEAR EAST', -0.19032480757326514,
    -0.3232636124824411, -0.04421734470810142, -0.27528113360605316,
    0.13255850810281325, -0.8054845935643491, 1.0119784924248225,
    0.6189182532646624, 1.0074863283776458, 0.20239896852403538,
    -0.043678728558593366, -0.13929748680369286, 1.3163604645710438,
    -0.3699637766938669, -0.6149300604558857, -0.854369594993175,
    0.263445277972641, 0.5712416961268142
]

In [25]:
test_countryDf = pd.DataFrame(np.array(test_country).reshape(1, -1), columns=countries.columns)
test_countryDf

,Country,Region,Population,Area,Pop_density,Coastline_ratio,Net_migration,Infant_mortality,GDP,Literacy,Phones_per_1000,Arable,Crops,Other,Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Test Country,NEAR EAST,-0.19032480757326514,-0.3232636124824411,-0.04421734470810142,-0.27528113360605316,0.13255850810281325,-0.8054845935643491,1.0119784924248225,0.6189182532646624,1.0074863283776458,0.20239896852403538,-0.043678728558593366,-0.13929748680369286,1.3163604645710438,-0.3699637766938669,-0.6149300604558857,-0.854369594993175,0.263445277972641,0.5712416961268142


In [26]:
def q4():
    # Retorne aqui o resultado da questão 4.
    numericTransformer = Pipeline(steps=[('Imputer', SimpleImputer(strategy='median')), ('StandardScaler', StandardScaler())])
    preProcessor = ColumnTransformer(transformers=[('num', numericTransformer, numeric_columns)])
    preProcessor.fit(countries)
    answer = preProcessor.transform(test_countryDf)
    answerDf = pd.DataFrame(answer, columns=countries.columns[2:])
    return round(answerDf['Arable'].values[0], 3)

In [27]:
q4()

-1.047

## Questão 5

Descubra o número de _outliers_ da variável `Net_migration` segundo o método do _boxplot_, ou seja, usando a lógica:

$$x \notin [Q1 - 1.5 \times \text{IQR}, Q3 + 1.5 \times \text{IQR}] \Rightarrow x \text{ é outlier}$$

que se encontram no grupo inferior e no grupo superior.

Você deveria remover da análise as observações consideradas _outliers_ segundo esse método? Responda como uma tupla de três elementos `(outliers_abaixo, outliers_acima, removeria?)` ((int, int, bool)).

In [43]:
def q5():
    # Retorne aqui o resultado da questão 4.
    quartil1 = countries['Net_migration'].quantile(q=0.25)
    quartil3 = countries['Net_migration'].quantile(q=0.75)
    iqr = quartil3-quartil1
    lowerLimit = quartil1-1.5*iqr
    upperLimit = quartil3+1.5*iqr
    lowerOutliers = countries[countries['Net_migration'] < lowerLimit]['Net_migration'].count()
    upperOutliers = countries[countries['Net_migration'] > upperLimit]['Net_migration'].count()
    return (lowerOutliers, upperOutliers, False)

In [48]:
# Calculando os quantis e o intervalo interquartil
quartil1 = countries['Net_migration'].quantile(q=0.25)
quartil3 = countries['Net_migration'].quantile(q=0.75)
iqr = quartil3-quartil1
iqr

1.9249999999999998

In [47]:
# Calculando os limites inferior e superior de acordo com o método do boxplot
lowerLimit = quartil1-1.5*iqr
upperLimit = quartil3+1.5*iqr
(lowerLimit, upperLimit)

(-3.8149999999999995, 3.885)

In [31]:
# Contando o número de outiliers no limite inferior e superior
lowerOutliers = countries[countries['Net_migration'] < lowerLimit]['Net_migration'].count()
upperOutliers = countries[countries['Net_migration'] > upperLimit]['Net_migration'].count()
(lowerOutliers, upperOutliers)

(24, 26)

In [32]:
# print do pontos no limite inferior
countries[countries['Net_migration'] < lowerLimit]['Net_migration']

1      -4.93
3     -20.71
7      -6.15
9      -6.47
13     -4.90
30     -4.58
37    -12.07
56    -13.87
59     -8.58
75     -4.70
80     -8.37
81    -13.92
102    -4.92
130    -6.04
135    -4.87
136   -20.99
172    -7.11
174    -4.86
175    -7.64
176   -11.70
182    -5.69
193    -8.81
204   -10.83
220    -8.94
Name: Net_migration, dtype: float64

In [33]:
# print dos pontos no limite superior
countries[countries['Net_migration'] > upperLimit]['Net_migration']

0      23.06
4       6.60
6      10.76
11      3.98
28     10.01
36      5.96
38     18.75
70      6.27
91      5.24
98      4.99
99      5.36
105     6.59
111    14.18
119     4.85
121     8.97
122     4.86
134     6.78
138     7.75
149     4.05
153     9.61
166    16.29
177    10.98
184    11.53
188     5.37
196     4.05
208    11.68
Name: Net_migration, dtype: float64

In [44]:
q5()

(24, 26, False)

## Questão 6
Para as questões 6 e 7 utilize a biblioteca `fetch_20newsgroups` de datasets de test do `sklearn`

Considere carregar as seguintes categorias e o dataset `newsgroups`:

```
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)
```


Aplique `CountVectorizer` ao _data set_ `newsgroups` e descubra o número de vezes que a palavra _phone_ aparece no corpus. Responda como um único escalar.

In [35]:
def q6():
    # Retorne aqui o resultado da questão 4.
    vectorizer = CountVectorizer()
    newsgroupCv = vectorizer.fit_transform(newsgroup.data)
    phone = vectorizer.vocabulary_['phone']
    return newsgroupCv.toarray().sum(axis=0)[phone]

In [36]:
# Fazendo download do dataset da classe datasets da biblioteca do scikit-learn
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)

In [37]:
# Transformando o texto aplicando a classe CountVectorizer e assim contando a frequência de cada palavra no dataset
vectorizer = CountVectorizer()
newsgroupCv = vectorizer.fit_transform(newsgroup.data)
phone = vectorizer.vocabulary_['phone']
newsgroupCv.toarray().sum(axis=0)[phone]

213

## Questão 7

Aplique `TfidfVectorizer` ao _data set_ `newsgroups` e descubra o TF-IDF da palavra _phone_. Responda como um único escalar arredondado para três casas decimais.

In [38]:
def q7():
    # Retorne aqui o resultado da questão 4.
    vectorizer = TfidfVectorizer()
    x = vectorizer.fit_transform(newsgroup.data)
    index = vectorizer.vocabulary_['phone']
    return round(x.toarray().sum(axis=0)[index], 3)

In [39]:
# Transformando o texto agora aplicando a classe TfidfVectorizer, além de considerar a frequência também considerá
# a relevância da palavra nos dados por inteiro
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(newsgroup.data)
index = vectorizer.vocabulary_['phone']
x.toarray().sum(axis=0)[index]

8.88774594667355

In [40]:
# Criando um dataset com as palavras e seus respectivo TF-IDF e ordenando do maior para o menor
cols = [*vectorizer.vocabulary_.keys()]
x.toarray().sum(axis=0)
dfAux = pd.DataFrame(x.toarray().sum(axis=0), index=cols)
dfAux.sort_values(by=0, ascending=False)[:10]

,0
groverc,159.222010
flurries,94.817501
meted,79.576625
keyring,71.229380
solberg,64.569538
marginally,60.838665
kicking,60.147179
kaschke,55.754465
utils820,53.255324
shopping,52.660130
